In [54]:
%load_ext autoreload
%autoreload 2

import sys
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from UNet import UNet, UNet_encoder
from ZUNet_v1 import ZUNet_v1, ZUNet_v2
from dataloader import TE_loader
import torch
import nibabel as nib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
    infer_path = "data/ProjSubjList.in"
    infer_list = pd.read_csv(infer_path)
    parameter_path = (
        "/data1/inqlee0704/silicosis/RESULTS/UNet_64_20211002/lung_UNet.pth"
        # "/data1/inqlee0704/silicosis/RESULTS/ZUNet_64_lung_20211001/ZUNet_lung.pth"
    )
    # parameter_path = '/home/inqlee0704/src/DL/airway/RESULTS/Recursive_UNet_v2_20201216/model.pth'
    # model = UNet()
    model = UNet()
    # model.load_state_dict(torch.load(parameter_path))

In [56]:
model_encoder = UNet_encoder()

In [57]:
input = torch.rand(1,1,512,512)
out = model_encoder(input)

In [58]:
model_dict = model.state_dict()

# 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict) 
# 3. load the new state dict
model.load_state_dict(pretrained_dict)

torch.Size([1, 1])